In [1]:
'''
This Code would use the generated model to create a real time prediction of gesture and assign the calss accordingly 
'''
import pickle

import cv2
import mediapipe as mp
import numpy as np

# Loading the created ML model
modelDict = pickle.load(open('./model.p', 'rb'))
model = modelDict['model']

# Choosing of video camera
cap = cv2.VideoCapture(0)

# Storing all mp methods/object into variables 
mpHands = mp.solutions.hands
mpDrawing = mp.solutions.drawing_utils
mpDrawingStyles = mp.solutions.drawing_styles

# Creatting instance of hand capturing method
hands = mpHands.Hands(static_image_mode=True, min_detection_confidence=0.3)

# Choosing name to be displayed for each class
labels_dict = {0: 'Rock', 1: 'paper', 2: 'scissor'}

# Starting the loop
while True:

    data_aux = []
    x_ = []
    y_ = []

    # Reading a frame
    ret, frame = cap.read()

    # Storing Height and width of the frame
    H, W, _ = frame.shape

    # Converting into RGB vectors
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

    # Segmentation of the hand from the whole image
    results = hands.process(frame_rgb)

    # If multiple hands are detected iterate for each hand
    if results.multi_hand_landmarks:
        for hand_landmarks in results.multi_hand_landmarks:
            
            # Drawing a rectangle box over each hand
            mpDrawing.draw_landmarks(
                frame,  # image to draw
                hand_landmarks,  # model output
                mpHands.HAND_CONNECTIONS,  # hand connections
                mpDrawingStyles.get_default_hand_landmarks_style(),
                mpDrawingStyles.get_default_hand_connections_style())

        # Checking for multiple hand location
        for hand_landmarks in results.multi_hand_landmarks:
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y

                # Storing the hand locations in lists
                x_.append(x)
                y_.append(y)
                
            # Calculating the distance from the lowest point
            for i in range(len(hand_landmarks.landmark)):
                x = hand_landmarks.landmark[i].x
                y = hand_landmarks.landmark[i].y
                data_aux.append(x - min(x_))
                data_aux.append(y - min(y_))

        # Generating box vertex coordinates wwith the min value
        x1 = int(min(x_) * W) - 10
        y1 = int(min(y_) * H) - 10

        x2 = int(max(x_) * W) - 10
        y2 = int(max(y_) * H) - 10

        # Predicting using the measured array of data
        prediction = model.predict([np.asarray(data_aux)])

        # Looking up for label with the predicted class in the initialized dictionary
        predicted_character = labels_dict[int(prediction[0])]

        # Generation of rectangular frame
        cv2.rectangle(frame, (x1, y1), (x2, y2), (0, 0, 0), 4)

        # Displaying of text
        cv2.putText(frame, predicted_character, (x1, y1 - 10), cv2.FONT_HERSHEY_SIMPLEX, 1.3, (0, 0, 0), 3,
                    cv2.LINE_AA)

    # Command to show window which displays the captured images
    cv2.imshow('frame', frame)
    cv2.waitKey(1)

# Flushing of all used windows and cameras
cap.release()
cv2.destroyAllWindows()

KeyboardInterrupt: 